<a href="https://colab.research.google.com/github/Apple03244/Colaboratory/blob/main/Kaggle_TiTanic_Pycaret.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [39]:
import google.colab as colab
f=colab.files.upload()

Saving kaggle.json to kaggle (1).json


In [40]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
# Permission Warning 방지
!chmod 600 ~/.kaggle/kaggle.json

In [41]:
!kaggle competitions download -c titanic
! unzip /content/titanic.zip

titanic.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  /content/titanic.zip
replace gender_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


---

In [59]:
import pandas as pd
train=pd.read_csv('/content/train.csv')
test=pd.read_csv('/content/test.csv')
sub_mission=pd.read_csv('/content/gender_submission.csv')

## Medel Search
- use Pycaret

In [60]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [61]:
#!pip install pycaret

In [62]:
import pycaret.classification as pycla

### step 1. data setup

In [63]:
train_setup = pycla.setup(data=train,
    target='Survived',
    ignore_features=["Cabin","Name","PassengerId","Ticket"],
    ordinal_features={"Pclass":train["Pclass"].unique()},
    categorical_features=['Embarked',"Sex"],
    numeric_features=["Age","SibSp","Parch","Fare"],
    imputation_type="simple",
    numeric_imputation="knn",
    categorical_imputation='mode',
    normalize=True,
    normalize_method="minmax",
    n_jobs=-1,
    use_gpu=True)

[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Number of positive: 1, number of negative: 1
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Warning] Using sparse features with CUDA is currently not supported.
[LightGBM] [Info] Number of positive: 1, number of negative: 1
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset migh

,Description,Value
0,Session id,1336
1,Target,Survived
2,Target type,Binary
3,Original data shape,"(891, 12)"
4,Transformed data shape,"(891, 10)"
5,Transformed train set shape,"(623, 10)"
6,Transformed test set shape,"(268, 10)"
7,Ignore features,4
8,Ordinal features,2
9,Numeric features,4


[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Number of positive: 1, number of negative: 1
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Warning] Using sparse features with CUDA is currently not supported.
[LightGBM] [Info] Number of positive: 1, number of negative: 1
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset migh

### step 2. compare Models

In [64]:
best_classifier=pycla.compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
gbc,Gradient Boosting Classifier,0.8346,0.8702,0.7154,0.8347,0.7679,0.6407,0.6475,0.3350
xgboost,Extreme Gradient Boosting,0.8314,0.8646,0.7449,0.8034,0.7724,0.6388,0.6406,0.1980
rf,Random Forest Classifier,0.8281,0.8771,0.7408,0.7999,0.7675,0.6316,0.6343,0.4390
lightgbm,Light Gradient Boosting Machine,0.8233,0.8746,0.7324,0.7947,0.7611,0.6214,0.6238,0.4290
et,Extra Trees Classifier,0.8088,0.8515,0.7241,0.7683,0.7436,0.5916,0.5941,0.3840
ridge,Ridge Classifier,0.8026,0.0000,0.6908,0.7805,0.7295,0.5752,0.5812,0.1310
lda,Linear Discriminant Analysis,0.8010,0.8451,0.6908,0.7766,0.7280,0.5722,0.5777,0.1350
ada,Ada Boost Classifier,0.7992,0.8389,0.7237,0.7513,0.7359,0.5742,0.5758,0.2840
qda,Quadratic Discriminant Analysis,0.7977,0.8352,0.7574,0.7316,0.7425,0.5764,0.5785,0.1370
lr,Logistic Regression,0.7946,0.8423,0.6908,0.7609,0.7211,0.5595,0.5639,0.1470


Processing:   0%|          | 0/65 [00:00<?, ?it/s]

### step 3. Hyper parameter tuning

In [65]:
tuned_result=pycla.tune_model(estimator=best_classifier,fold=10,n_iter=100,
                 optimize="Accuracy",return_train_score=True,return_tuner=True)

Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 100 candidates, totalling 1000 fits


#### Result
- this model's 83% accuracy


### step 4. save models and predict

In [65]:
pycla.save_model(tuned_result[0],"best_model")
pycla.save_model(tuned_result[1],"tuned_best_model")

In [112]:
tuned_GBClassifier,tuned_GBClassifier_process=tuned_result

In [113]:
tuned_GBClassifier_process

RandomizedSearchCV(cv=StratifiedKFold(n_splits=10, random_state=None, shuffle=False),
                   error_score=nan,
                   estimator=Pipeline(memory=Memory(location=None),
                                      steps=[('numerical_imputer',
                                              TransformerWrapper(exclude=None,
                                                                 include=['Age',
                                                                          'SibSp',
                                                                          'Parch',
                                                                          'Fare'],
                                                                 transformer=KNNImputer(add_indicator=False,
                                                                                        copy=True,
                                                                                        keep_empty_features=False,
                                                                                        metric='nan_euclidean',
                                                                                        mi...
                                        'actual_estimator__n_estimators': [10,
                                                                           20,
                                                                           30,
                                                                           40,
                                                                           50,
                                                                           60,
                                                                           70,
                                                                           80,
                                                                           90,
                                                                           100,
                                                                           110,
                                                                           120,
                                                                           130,
                                                                           140,
                                                                           150,
                                                                           160,
                                                                           170,
                                                                           180,
                                                                           190,
                                                                           200,
                                                                           210,
                                                                           220,
                                                                           230,
                                                                           240,
                                                                           250,
                                                                           260,
                                                                           270,
                                                                           280,
                                                                           290,
                                                                           300],
                                        'actual_estimator__subsample': [0.2,
                                                                        0.25,
                                                                        0.3,
                                                                        0.35,
                                                                        0.4,
                          

In [114]:
# make final model
prediction=pycla.predict_model(tuned_GBClassifier,data=test)

In [117]:
prediction["prediction_label"].value_counts()

0    269
1    149
Name: prediction_label, dtype: int64

In [121]:
import sklearn.metrics as skmet
skmet.accuracy_score(y_true=sub_mission["Survived"],y_pred=prediction["prediction_label"])

0.8636363636363636